**Main**

In [ ]:
from pandas.io.formats.format import DataFrameFormatter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

df = pd.read_csv("BWP_rates.csv")

newdf = df
newdf['date_float'] = pd.to_datetime(df['ds']).astype(int) / 10**9 / 86400
newdf = df.drop(['ds'], axis=1)


newdf = newdf.values.reshape(-1, 1)

print(newdf)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
newdf = scaler.fit_transform(newdf)


# split into train and test sets
train_data, test_data = traintest(newdf)

# convert an array of values into a dataset matrix
def create_dataset(newdf, back=1):
  dataX, dataY = [], []  
  for i in range(len(newdf)-back-1):
    a = newdf[i:(i+back), 0]
    dataX.append(a)
    dataY.append(newdf[i + back, 0])
    return np.array(dataX), np.array(dataY)

# reshape into X=t and Y=t+1
back = 1
trainX, trainY = create_dataset(train_data, back)
testX, testY = create_dataset(test_data, back)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)


# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

# shift train predictions for plotting
trainPredictPlot = np.empty_like(newdf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[back:len(trainPredict)+back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(newdf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(back*2)+1:len(newdf)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(newdf))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

**Function to run the training and testing data**

In [ ]:
def traintest(newdf):

    # Split the data into training and testing sets
    train_data, test_data = train_test_split(newdf, test_size=0.2, shuffle=False, random_state=9)

    # Print the number of rows in each set
    print(f'Training set contains {len(train_data)} rows.')
    print(f'Testing set contains {len(test_data)} rows.')

    return train_data, test_data

print(traintest(newdf))